In [ ]:
from tensorflow.keras.datasets import mnist #케라스(tensorflow >2.0)에서 받을수 있는 데이터 셋
#from tensorflow.keras.utils import np_utils
from tensorflow.python.keras.utils.np_utils import to_categorical #np_utils 모듈 수정됨, 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D #Dropout, Flatten, Conv2D 등 import
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping #모델저장, 과적합 방지

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import cv2

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [ ]:
# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() #데이터 불러오기
#총7만개 자료, 6만개 Train데이터 1만개 Test데이터
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
#28, 28 , 1 --> 28*28픽셀 흑백영상이라 채널이 1 
#0 ~ 255로 분포되어 있는 픽셀값을 0~1사이의 실수값으로 변경하기 위해 255로 나눔
#파이썬에서 / 는 소수점까지 구해진다는 것을 잊으면 안됨 
#정수로 나누고 싶다면 //
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
Y_train = to_categorical(Y_train) #손글씨 결과를 0~9까지 표현하는데 원-핫인코딩 결과로 리턴
Y_test = to_categorical(Y_test) #테스트 결과를 원-핫인코딩함, 이유는 텐서플로우에서 효율적인 계산을 위해서

In [ ]:
model = load_model('./dataset/MNIST_학습된 모델/09-0.0266.hdf5') # 모델을 새로 불러옴

print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))  # 불러온 모델로 테스트 실행

In [ ]:
cap = cv2.VideoCapture(0)
frmae_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
              int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

In [ ]:
while True:
    retval, frame = cap.read()
    if not retval:
        break
    
    cv2.imshow('frame', frame)
    
    
    key = cv2.waitKey(25)
    if key == 27:       # 27 Esc
        break
    elif key == 32:     # 32 스페이스바
        cv2.imwrite('./hand.png', frame)
        break
        

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread('./hand.png', cv2.IMREAD_GRAYSCALE)

ret, blmage = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(15,15))
erode = cv2.erode(blmage, kernel, 7)

dst = erode[100:450, 100:500].copy()

resize_img = cv2.resize(dst, (28, 28), fx=1, fy=1,interpolation = cv2.INTER_AREA)
last_img = 255 - resize_img.copy()

plt.imshow(last_img, cmap= 'Greys')
plt.show()

In [ ]:
img3 = last_img.reshape(1, 28, 28, 1) / 255
model.predict_classes(img3)